# Exception Handling

Note: The Jupyter Run All command stops after each exception. Naturally for this subject, there will be a number of exceptions in this chapter, thus notebook execution needs to be continued manually after each exception.

## Causes of Exceptions

In [1]:
1 / 0 # causes ZeroDivisionError

ZeroDivisionError: division by zero

In [2]:
raise Exception('test exception') # explicitly raises an exception

Exception: test exception

Exceptions can either originate from commands (here a zero division) or are explicitly raised.

If an exception is raised, the current function is exited and it is returned to the calling function, until the exception is either catched or the progam is exited to the (Python or system) shell.

## Catching Exceptions

In [3]:
try:
    1/0
except ZeroDivisionError as e:# basic pattern
    print(e)

division by zero


Basic pattern: catching one specific exception type in the try block and print the error message

In [4]:
def one_over(x):
    y = float('nan')
    try:
        y = 1/x
    except ZeroDivisionError:
        print('invalid division by 0')
    except TypeError:
        print('x must be numeric')
    except Exception as e:
        print('something else happened')
        print(e)
    finally:
        return y

In [5]:
one_over(2)

0.5

In [6]:
one_over(0)

invalid division by 0


nan

In [7]:
one_over('hello')

x must be numeric


nan

Explicit catching of multiple error types allows different actions for each.

Exception is the base type of all exceptions, therefore the last except statement catches all possible exceptions, but is only executed if none of the previous except statements matched.

The finally block is always executed, regardless if there was an error in the try block or not.

In [8]:
class Number42Exception(Exception):
    pass

Custom exceptions can be created by inheritance of the existing exceptions (usually Exception class).

In [9]:
def one_over_not_42(x):
    try:
        if x == 42:
            raise Number42Exception('input 42 is not allowed')
        return 1/x
    except (ZeroDivisionError, TypeError) as e:
        print(e)
        return float('nan')
    except Exception as e:
        print(e)
        raise

In [10]:
one_over_not_42(0)

division by zero


nan

In [11]:
one_over_not_42('hello')

unsupported operand type(s) for /: 'int' and 'str'


nan

If a tuple of exceptions is given after *except*, any of them is catched.

In [12]:
one_over_not_42(42) # raises Exception

input 42 is not allowed


Number42Exception: input 42 is not allowed

The *raise* command in an except block re-raises the catched exception.

## Usage

* Do not be afraid to use (catched) exceptions as part of the normal program flow. An exception is not nessesarily an error, see e.g. the StopIteration exception of the Python standard library.
* It is not required to catch every exception. If there is something broken in the program it is OK that it crashes. Exceptions regarding user input, etc. however should be catched.

## Antipatterns

In [13]:
x = 0
try:
    1/0
except:
    pass

Probably __the__ worst thing which could be done: all exceptions are ignored and no information about it is stored. Happy debugging!

In [14]:
x = 0
try:
    1/0
except:
    print('zero division error')

zero division error


All error types are catched, the message however assumes a zero division error.
It is recommended to catch only specific errors.

In [15]:
x = 0
try:
    1/0
except Exception:
    print('some other error')
except ZeroDivisionError:
    print('zero division error')

some other error


Wrong order of except statements: catch the more specific exception types first!

# Logging

In [16]:
import logging

In [17]:
# logging to console
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
#logging to file (usually for applications, not for notebook usage)
#logging.basicConfig(level=logging.INFO, 
#                   format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', 
#                   filename='logfile.log', filemode='w')

In [18]:
logging.critical('critical error %i', 42)
logging.error('this is an error with {} consequences'.format('dire'))
logging.warning(f'this is a warning, countdown: {", ".join(str(10-i) for i in range(10))}')
logging.info('just for information: costs are %.2f EUR', 42.42)
logging.debug('debug info currently not shown')

2019-06-14 19:06:33,351 root         CRITICAL critical error 42
2019-06-14 19:06:33,355 root         ERROR    this is an error with dire consequences
2019-06-14 19:06:33,357 root         WARNING  this is a warning, countdown: 10, 9, 8, 7, 6, 5, 4, 3, 2, 1
2019-06-14 19:06:33,358 root         INFO     just for information: costs are 42.42 EUR


Logging entries for different severities can be added. The lowest severity level which is shown in the log is specified by the level keyword.

The content of variables can be added analogue to standard strings, i.e. with the % style syntax, string formatting or f-strings (recommended for Python >= 3.6).

Logging can be done to different targets, e.g. standard-out (like here) or a log file.

In [20]:
logger = logging.getLogger(__name__)
logger

<Logger __main__ (INFO)>

In [21]:
logger.info('just for information')

2019-06-14 19:07:02,483 __main__     INFO     just for information


Using *logging.severity* uses the root logger. For larger programs it is recommended to define in each module a separate logger using the statements above. This has the advantage that the module name itself is logged.

Note that if the loggers are defined after *logging.basicConfig*, they use the config defined there.

There are a lot of additional config possibilities for the Python logging module, which are only required in certain cases, see https://docs.python.org/3/library/logging.html.

In [22]:
def div(x, y):
    return x/y
try:
    div(1, 0)
except ZeroDivisionError:
    logger.exception('surprise: 1/0 caused an error')

2019-06-14 19:07:05,104 __main__     ERROR    surprise: 1/0 caused an error
Traceback (most recent call last):
  File "<ipython-input-22-2d43593ccf66>", line 4, in <module>
    div(1, 0)
  File "<ipython-input-22-2d43593ccf66>", line 2, in div
    return x/y
ZeroDivisionError: division by zero


*logging.exception* can be called in an *except* block and logs an error including the complete stack trace (unlike *logging.error*, which logs the error, but not the stack trace).

The logger of the standard Python library is easy to use and very powerful. It is strongly discouraged to implement a custom logger or to use a third party one.

Author: Benjamin Lungwitz